In [ ]:
#notebook for processing soccer player transfer data
#this will handle two functions:

# 1. scrape data from https://www.soccernews.com/soccer-transfers/german-bundesliga-transfers/ to .csv
# 2. reformat .csv to format required by tableau

# the format required by tableau to plot lines on a map is

# origin-destination | station | path ID

# each path has two entries
# we also need to find longitude and latitude here, using Nominatim
# because Tableau will find cities in the US by default (e.g. Paris, Texas)
# and these will be incorrect

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import requests
import urllib.request
from bs4 import BeautifulSoup
import time

In [ ]:
list(set(list(dat["From"].values) + list(dat["To"].values) ))

In [193]:
team_to_city_dict = {
                "AC Milan":"Milan",\
                "Ajax":"Amsterdam",\
                "Arsenal":"London",\
                "Aston Villa":"Aston",\
                "Atletico Madrid":"Madrid",\
                "ADO Den Haag": "Den Haag",\
                "AS Roma":"Rome",\
                "Bayer Leverkusen":"Leverkusen",\
                "Bayern Munich":"Munich",\
                "Bristol City":"Bristol",\
                "Boavista":"Porto",\
                "Borussia Dortmund":"Dortmund",\
                "Bor. Dortmund":"Dortmund",\
                "BSC Young Boys":"Bern",\
                "Celtic":"Glasgow",\
                "Club Brugge":"Bruges",\
                "FC Copenhagen":"Copenhagen",\
                "Crvena Zvezda":"Belgrade",\
                "Crystal Palace":"London",\
                "Dynamo Dresden":"Dresden",\
                "Eintracht Frankfurt":"Frankfurt",\
                "Espanyol":"Barcelona",\
                "Everton":"Liverpool",\
                "FC Barcelona":"Barcelona",\
                "FC Dallas":"Dallas",\
                "FC Koln":"Koln",\
                "FC Luzern":"Lucerne",\
                "Feyenoord":"Rotterdam",\
                "Fortuna Dusseldorf":"Dusseldorf",\
                "Grasshoppers":"Zurich",\
                "Hannover 96":"Hannover",\
                "Hertha Berlin":"Berlin",\
                "Hoffenheim":"Heidelberg",\
                "Internacional":"Porto Alegre",\
                "Instituto":"Cordoba",\
                "Ituano":"Sao Paolo",\
                "Jahn Regensburg":"Regensburg",\
                "Juventus":"Turin",\
                "Kasimpasa":"Istanbul",\
                "KRC Genk":"Genk",\
                "KV Mechelen":"Mechelen",\
                "LASK Linz":"Linz",\
                "Lazio":"Rome",\
                "Leeds United":"Leeds",\
                "Mainz 05":"Mainz",\
                "Manchester City":"Manchester",\
                "Man United":"Manchester",\
                "Newcastle United":"Newcastle",\
                "Nordsjaelland":"Copenhagen",\
                "Norwich City":"Norwich",\
                "Ostersunds":"Ostersund",\
                "Partick Thistle":"Glasgow",\
                "PEC Zwolle":"Zwolle",\
                "PSG":"Paris",\
                "PSV Eindhoven":"Eindhoven",\
                "QPR":"London",\
                "RB Leipzig":"Leipzig",\
                "RB Salzburg":"Salzburg",\
                "Real Betis":"Seville",\
                "Real Madrid":"Madrid",\
                "Rennais":"Rennes",\
                "Sampdoria":"Genoa",\
                "Schalke 04":"Gelsenkirchen",\
                "Sheffield United":"Sheffield",\
                "Sporting Lisbon":"Lisbon",\
                "Stade Rennais":"Rennais",\
                "Standard Liege":"Liege",\
                "Stoke":"Stoke-on-Trent",\
                "Swansea City":"Swansea",\
                "Tottenham":"London",\
                "Union Berlin":"Berlin",\
                "Waasland-Beveren":"Beveren",\
                "Werder Bremen":"Bremen",\
                "West Brom":"West Bromwich",\
                "West Ham United":"London",\
                "Wolves":"Wolverhampton",\
                "Young Boys":"Bern"}

In [ ]:
regions = ["Europe", "Germany", "France", "UK", "Italy", "Austria", "Serbia", "Denmark", "Brazil", "Argentina"]

In [ ]:
location = geolocator.geocode("Guingamp, France")

In [ ]:
location

In [202]:
#convert team names to city name
#then convert city name to (latitude, longitude) using Nominatim
def team_to_location(team, geolocator):
    if team in team_to_city_dict: city = team_to_city_dict[team]
    else: city = team
    entry = {}
    entry['Team'] = team
    entry['City'] = city
    for region in regions:
        location = geolocator.geocode(city + ", " + region, timeout=10)
        time.sleep(2)
        if(not location is None): break
    if(location is None):
        print("Warning - could not find this city anywhere: " + city)
        return None
    else:
        entry['Latitude'] = location.latitude
        entry['Longitude'] = location.longitude
        entry['In UK'] = -6 <= location.longitude and location.longitude <= 2\
        and 50 <= location.latitude and location.latitude <= 56
        entry['In Germany'] = 6 <= location.longitude and location.longitude <= 15\
        and 49 <= location.latitude and location.latitude <= 55
    return entry

In [200]:
def convert_for_tableau(filename, league):
    
    no_ext = lambda f : (".").join(f.split(".")[:-1])
    
    geolocator = Nominatim(user_agent="soccer_transfers")
    
    dat_orig = pd.read_csv(filename)
    dat_conv = []
    #generate dataframe from list of dicts
    
    for i,row in dat_orig.iterrows():
        print("Processing entry %00d/%00d"%(i+1, dat_orig.shape[0]))
        #.csv from excel can have empty rows
        if(not isinstance(row["Player Name"], str)):
            break
        #ignore some incomplete entries
        if(not all([isinstance(row[field], str) for field in ["From", "To"]])):
            continue
        if(any([row[field]=="nan" for field in ["From", "To"]])):
            continue
            
        fromloc = team_to_location(row["From"], geolocator)
        toloc = team_to_location(row["To"], geolocator)
        if(fromloc is None or toloc is None): 
            continue
            
        from_foreign = (not fromloc['In UK'] and league=="EPL") or\
                        (not fromloc['In Germany'] and league=="Bundesliga")
        to_foreign   = not from_foreign and (not toloc['In UK'] and league=="EPL") or\
                        (not toloc['In Germany'] and league=="Bundesliga")
        #ensure that these are mutually exclusive, they should normally be but we could have geocoding errors 
        
        category     = 0 + 1*from_foreign + 2*to_foreign
            
            
        #set common fields
        orig_entry = {
                    "Player Name":row["Player Name"],\
                    "Player Position":row["Player Position"],\
                    "Transfer Details":row["Transfer Details"],\
                    "Path ID":no_ext(filename)+"-"+str(i),\
                    "League":league,\
                    "Category":category
                     }
        
        dest_entry = orig_entry.copy()
        
        #set fields that differ between origin and destination entry
        orig_entry["Origin-Destination"] = "Origin"
        dest_entry["Origin-Destination"] = "Destination"
        
        

        orig_entry.update(fromloc)       
        dest_entry.update(toloc)
        dat_conv.append(orig_entry)
        dat_conv.append(dest_entry)
        
    
        
    dat_conv = pd.DataFrame(dat_conv)
    dat_conv.to_csv(no_ext(filename)+"_tableau.csv", index=False)

In [195]:
def scrape_to_csv(url, output_filename):
    article = requests.get(url)
    soup = BeautifulSoup(article.text, "html.parser")
    text = ""
    tags = soup.findAll("tr")
    dict_list = []
    for i in range(len(tags)):
        if(i==0): continue
        t = tags[i]
        s = BeautifulSoup(t.text, "html.parser")
        s2 = str(s).split('\n')
        s2 = [x for x in s2 if len(x) > 0] 
        if(s2[0]=="When"): break
        #this signals that we reached the end of the table
        #there are some record-setting expensive transfers after it
        #do not include these
        entry = {}
        entry["Date"] = s2[0] + ' ' + '2019'
        namepos = s2[1].split()
        entry["Player Position"] = namepos[-1]
        entry["Player Name"] = " ".join(namepos[:-1])
        entry["From"] = s2[2]
        entry["To"] = s2[3]
        entry["Transfer Details"] = s2[4]
        dict_list.append(entry)
    pd.DataFrame(dict_list).to_csv(output_filename)

In [ ]:
scrape_to_csv("https://www.soccernews.com/soccer-transfers/german-bundesliga-transfers/", "bundesliga.csv")

In [ ]:
convert_for_tableau("bundesliga.csv", "Bundesliga")

In [ ]:
scrape_to_csv("https://www.soccernews.com/soccer-transfers/english-premier-league-transfers/", "epl.csv")

In [ ]:
dat = pd.read_csv("epl.csv")

In [203]:
convert_for_tableau("epl.csv", "EPL")

Processing entry 1/110
Processing entry 2/110
Processing entry 3/110
Processing entry 4/110
Processing entry 5/110
Processing entry 6/110
Processing entry 7/110
Processing entry 8/110
Processing entry 9/110
Processing entry 10/110
Processing entry 11/110
Processing entry 12/110
Processing entry 13/110
Processing entry 14/110
Processing entry 15/110
Processing entry 16/110
Processing entry 17/110
Processing entry 18/110
Processing entry 19/110
Processing entry 20/110
Processing entry 21/110
Processing entry 22/110
Processing entry 23/110
Processing entry 24/110
Processing entry 25/110
Processing entry 26/110
Processing entry 27/110
Processing entry 28/110
Processing entry 29/110
Processing entry 30/110
Processing entry 31/110
Processing entry 32/110
Processing entry 33/110
Processing entry 34/110
Processing entry 35/110
Processing entry 36/110
Processing entry 37/110
Processing entry 38/110
Processing entry 39/110
Processing entry 40/110
Processing entry 41/110
Processing entry 42/110
P

In [198]:
convert_for_tableau("bundesliga.csv", "Bundesliga")

Processing entry 0/116
Processing entry 1/116
Processing entry 2/116
Processing entry 3/116
Processing entry 4/116
Processing entry 5/116
Processing entry 6/116
Processing entry 7/116
Processing entry 8/116
Processing entry 9/116
Processing entry 10/116
Processing entry 11/116
Processing entry 12/116
Processing entry 13/116
Processing entry 14/116
Processing entry 15/116
Processing entry 16/116
Processing entry 17/116
Processing entry 18/116
Processing entry 19/116
Processing entry 20/116
Processing entry 21/116
Processing entry 22/116
Processing entry 23/116
Processing entry 24/116
Processing entry 25/116
Processing entry 26/116
Warning - could not find this city anywhere: Rennais
Processing entry 27/116
Processing entry 28/116
Processing entry 29/116
Processing entry 30/116
Processing entry 31/116
Processing entry 32/116
Processing entry 33/116
Processing entry 34/116
Processing entry 35/116
Processing entry 36/116
Processing entry 37/116
Processing entry 38/116
Processing entry 39/1